In [213]:
# Get libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

In [209]:
# Parse data file and post-process
filename = "data/jp_2020_name_vote.csv"

df = pd.read_csv(filename)

df.drop(columns=df.columns[11],axis=1,inplace=True)
df.drop(columns=df.columns[0:2],axis=1,inplace=True)
col_names = [(orig,re.search("\[.*\]",orig).group(0).strip("[]")) for orig in df.columns]
rename_cols = dict()
rename_cols.update(col_names)
df.rename(columns=rename_cols, inplace=True)
df = df.applymap(lambda x: int(x[-1]))

ranked_votes = df.to_numpy()
num_voters = len(df.index)
num_candidates = len(df.columns)
required_majority = int(np.ceil(num_voters/2))
remaining_contenders = df.columns
contenders = remaining_contenders

print("Number of voters: ", num_voters)
print("Number of candidates: ", num_candidates)
print("Required votes to win: ", required_majority)
print(df)

Number of voters:  19
Number of candidates:  9
Required votes to win:  10
    Argonauts  Sea dragons  Axolotls  Coelacanths  Pteropods  Tardigrades  \
0           2            4         1            3          6            7   
1           2            9         8            1          5            7   
2           2            1         6            3          7            4   
3           1            8         6            5          2            3   
4           1            6         9            2          4            7   
5           7            4         9            6          2            8   
6           3            4         8            1          2            6   
7           5            4         2            7          3            6   
8           1            2         9            5          4            8   
9           7            6         2            3          9            5   
10          2            8         3            5          6            4   
11

In [210]:
# Helper functions
def get_weakest_candidate(df):
    possible_weakest = df.columns
    
    for rank in range(1,len(df.columns)+1):
        possible_weakest = possible_weakest[(df[possible_weakest]==rank).sum()==(df[possible_weakest]==rank).sum().min()]
        if len(possible_weakest) == 1:
            break
    if len(possible_weakest) != 1:
        print("Something went wrong: there are ", len(possible_weakest), " weakest candidates.")
    return possible_weakest

def redistribute_votes(df, weakest_candidate):
    df_updated = df.drop(weakest_candidate[0], axis=1)
    df_updated[df_updated.apply(lambda col: col > df[weakest_candidate[0]])] -= 1
    return df_updated

In [221]:
# Do Run-off voting
df_running = df.copy()

for round in range(num_candidates):
    print("Round: ", round)
    #print("Votes: ", df_running)
    weakest_candidate = get_weakest_candidate(df_running)
    print("Weakest candidate: ", weakest_candidate[0])
    df_running = redistribute_votes(df_running, weakest_candidate)
    if ((df_running==1).sum() > required_majority).any():
        break

print("Final showdown: \n", (df_running==1).sum())


Round:  0
Weakest candidate:  Dinoflagellates
Round:  1
Weakest candidate:  Tardigrades
Round:  2
Weakest candidate:  Sea dragons
Round:  3
Weakest candidate:  Axolotls
Round:  4
Weakest candidate:  Pteropods
Round:  5
Weakest candidate:  Ctenophores
Round:  6
Weakest candidate:  Coelacanths
Final showdown: 
 Argonauts                              14
Antarctic Intermediate Water (AAIW)     5
dtype: int64
